In [ ]:
import socket
if socket.gethostname() == 'dlm':
  %env CUDA_DEVICE_ORDER=PCI_BUS_ID
  %env CUDA_VISIBLE_DEVICES=3

In [1]:
import os
import sys
import re
import collections
import functools
import itertools
import requests, zipfile, io
import pickle
import copy

import pandas
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import sklearn
import sklearn.decomposition
import sklearn.metrics
import networkx

import torch
import torch.nn as nn

lib_path = 'I:/code'
if not os.path.exists(lib_path):
  lib_path = '/media/6T/.tianle/.lib'
if not os.path.exists(lib_path):
  lib_path = '/projects/academic/azhang/tianlema/lib'
if os.path.exists(lib_path) and lib_path not in sys.path:
  sys.path.append(lib_path)
  
from dl.models.basic_models import *
from dl.utils.visualization.visualization import *
from dl.utils.outlier import *
from dl.utils.train import *
from autoencoder.autoencoder import *
from vin.vin import *
from dl.utils.utils import get_overlap_samples, filter_clinical_dict, get_target_variable
from dl.utils.utils import get_shuffled_data, target_to_numpy, discrete_to_id, get_mi_acc
from dl.utils.utils import get_label_distribution, normalize_continuous_variable

%load_ext autoreload
%autoreload 2


use_gpu = True
if use_gpu and torch.cuda.is_available():
  device = torch.device('cuda')
  print('Using GPU:)')
else:
  device = torch.device('cpu')
  print('Using CPU:(')

Using CPU:(


In [2]:
# neural net models include nn (mlp), resnet, densenet; another choice is ml (machine learning)
# model_type, dense, residual are dependent
model_type = 'resnet'
dense = False
residual = True
hidden_dim = [100, 100]
train_portion = 0.7
val_portion = 0.1
test_portion = 0.2
num_train_types = -1 # -1 means not used
num_val_types = -1
num_test_types = -1 # this will almost never be used 
num_sets = 10
num_folds = 10 # no longer used anymore
sel_set_idx = 0
cv_type = 'instance-shuffle' # or 'group-shuffle'; cross validation shuffle method
sel_disease_types = 'all'
# The number of total samples and the numbers for each class in selected disease types must >=
min_num_samples_per_type_cls = [100, 0]
# if 'auto-search', will search for the file first; if not exist, then generate random data split
# and write to the file;
# if string other than 'auto-search' is provided, assume the string is a proper file name, 
# and read the file;
# if False, will generate a random data split, but not write to file 
# if True will generate a random data split, and write to file
predefined_sample_set_file = 'auto-search' 
target_variable = ['PFI', 'DFI', 'PFI.time'] # To do: target variable can be a list (partially handled)
target_variable_type = ['discrete', 'discrete', 'continuous'] # or 'continuous' real numbers
target_variable_range = [[0,1],[0,1],[0,float('Inf')]]
data_type = ['gene', 'methy', 'rppa', 'mirna']
normal_transform_feature = True
additional_vars = ['age_at_initial_pathologic_diagnosis', 'gender', 'ajcc_pathologic_tumor_stage']
additional_var_types = ['continuous', 'discrete', 'discrete']
additional_var_ranges = [[0, 100], ['MALE', 'FEMALE'], 
                         ['I/II NOS', 'IS', 'Stage 0', 'Stage I', 'Stage IA', 'Stage IB', 
                          'Stage II', 'Stage IIA', 'Stage IIB', 'Stage IIC', 'Stage III',
                          'Stage IIIA', 'Stage IIIB', 'Stage IIIC', 'Stage IV', 'Stage IVA',
                          'Stage IVB', 'Stage IVC', 'Stage X']]
randomize_labels = False
lr = 5e-4
weight_decay = 1e-4
num_epochs = 100
reduce_every = 500
show_results_in_notebook = True

## Prepare data

In [3]:
result_folder = 'results'
data_split_idx_folder = f'{result_folder}/data_split_idx'
project_folder = '../../pan-can-atlas' # on dlm or ccr
print_stats = True
if not os.path.exists(project_folder):
  project_folder = 'F:/TCGA/Pan-Cancer-Atlas' # on my own desktop
filepath = f'{project_folder}/data/processed/combined2.pkl'
with open(filepath, 'rb') as f:
  data = pickle.load(f)
  patient_clinical = data['patient_clinical']
  feature_mat_dict = data['feature_mat_dict']
  feature_interaction_mat_dict = data['feature_interaction_mat_dict']
  feature_id_dict = data['feature_id_dict']
  aliquot_id_dict = data['aliquot_id_dict']
#   sel_patient_ids = data['sample_id_sel']
#   sample_idx_sel_dict = data['sample_idx_sel_dict']
#   for k, v in sample_idx_sel_dict.items():
#     assert [i[:12] for i in aliquot_id_dict[k][v]] == sel_patient_ids

if print_stats:
  for k, v in feature_mat_dict.items():
    print(f'feature_mat: {k}, max={v.max():.3f}, min={v.min():.3f}, '
          f'mean={v.mean():.3f}, {np.mean(v>0):.3f}')  
  for k, v in feature_interaction_mat_dict.items():
    print(f'feature_interaction_mat: {k}, max={v.max():.3f}, min={v.min():.3f}, '
          f'mean={v.mean():.3f}, {np.mean(v>0):.3f}') 
  for k, v in feature_id_dict.items():
    print(k, v.shape, v[0])
  for k, v in aliquot_id_dict.items():
    print(k, v.shape, v[0])

feature_mat: rppa, max=14.141, min=-7.869, mean=0.095, 0.516
feature_mat: mirna, max=11.813, min=0.000, mean=3.743, 1.000
feature_mat: gene, max=16.311, min=0.000, mean=8.412, 1.000
feature_mat: methy, max=1.000, min=0.000, mean=0.553, 1.000
feature_interaction_mat: rppa, max=1.000, min=0.000, mean=0.198, 0.277
feature_interaction_mat: mirna, max=1.000, min=0.010, mean=0.492, 1.000
feature_interaction_mat: gene, max=1.000, min=0.000, mean=0.050, 0.146
feature_interaction_mat: methy, max=1.000, min=0.000, mean=0.057, 0.127
rppa (189,) X1433EPSILON
mirna (662,) hsa-let-7a-2-3p
gene (4942,) A1BG
methy (4753,) cg00005847
rppa (7480,) TCGA-OR-A5J2-01A-21-A39K-20
mirna (9554,) TCGA-C4-A0F6-01A-11R-A10V-13
gene (9702,) TCGA-OR-A5J1-01A-11R-A29S-07
methy (10268,) TCGA-02-0001-01C-01D-0186-05


In [4]:
# select samples with required clinical variables
clinical_dict = filter_clinical_dict(target_variable, target_variable_type=target_variable_type, 
                                     target_variable_range=target_variable_range, 
                                     clinical_dict=patient_clinical)
if len(additional_vars) > 0:
  clinical_dict = filter_clinical_dict(additional_vars, target_variable_type=additional_var_types, 
                                       target_variable_range=additional_var_ranges, 
                                       clinical_dict=clinical_dict)

# select samples with feature matrix of given type(s)
if isinstance(data_type, str):
  sample_list = {s[:12] for s in aliquot_id_dict[data_type]}
  data_type_str = data_type
elif isinstance(data_type, (list, tuple)):
  sample_list = get_overlap_samples([aliquot_id_dict[dtype] for dtype in data_type], 
                                    common_list=None, start=0, end=12, return_common_list=True)
  data_type_str = '-'.join(sorted(data_type))
else:
  raise ValueError(f'data_type must be str or list/tuple, but is {type(data_type)}')
sample_list = set(sample_list).intersection(clinical_dict)

# select samples with given disease types
sel_disease_type_str = sel_disease_types # will be overwritten if it is a list
if isinstance(sel_disease_types, (list, tuple)):
  sample_list = [s for s in sample_list if clinical_dict[s]['type'] in sel_disease_types]
  sel_disease_type_str = '-'.join(sorted(sel_disease_types))
elif isinstance(sel_disease_types, str) and sel_disease_types!='all':
  sample_list = [s for s in sample_list if clinical_dict[s]['type'] == sel_disease_types]
else:
  assert sel_disease_types == 'all'
 
# For classification tasks with given min_num_samples_per_type_cls,
# only keep disease types that have a minimal number of samples per type and per class
# Reflection: it might be better to use collections.defaultdict(list) to store samples in each type
type_cnt = collections.Counter([clinical_dict[s]['type'] for s in sample_list])
if sum(min_num_samples_per_type_cls)>0 and (target_variable_type=='discrete' 
                                            or target_variable_type[0]=='discrete'):
  # the number of samples in each disease type >= min_num_samples_per_type_cls[0]
  type_cnt = {k: v for k, v in type_cnt.items() if v >= min_num_samples_per_type_cls[0]}
  disease_type_cnt = {}
  for k in type_cnt:
    # collections.Counter can accept generator
    cls_cnt = collections.Counter(clinical_dict[s][target_variable] 
                                  if isinstance(target_variable, str) 
                                  else clinical_dict[s][target_variable[0]] 
                                  for s in sample_list if clinical_dict[s]['type']==k)
    if all([v >= min_num_samples_per_type_cls[1] for v in cls_cnt.values()]):
      # the number of samples in each class >= min_num_samples_per_type_cls[1]
      disease_type_cnt[k] = dict(cls_cnt)
      print(k, disease_type_cnt[k])
  sample_list = [s for s in sample_list if clinical_dict[s]['type'] in disease_type_cnt]
sel_patient_ids = sorted(sample_list)
print(f'Selected {len(sel_patient_ids)} patients from {len(disease_type_cnt)} disease_types')

THCA {0.0: 240, 1.0: 24}
BLCA {0.0: 125, 1.0: 25}
BRCA {0.0: 666, 1.0: 62}
KIRP {0.0: 101, 1.0: 22}
STAD {1.0: 37, 0.0: 149}
LIHC {0.0: 62, 1.0: 68}
LUAD {1.0: 56, 0.0: 129}
COAD {0.0: 108, 1.0: 17}
LUSC {0.0: 136, 1.0: 56}
Selected 2083 patients from 9 disease_types


### Split data into training, validation, and test sets

In [5]:
predefined_sample_set_filename = (target_variable if isinstance(target_variable,str) 
                                else '-'.join(target_variable))
predefined_sample_set_filename += f'_{cv_type}'
if len(additional_vars) > 0:
  predefined_sample_set_filename += f"_{'-'.join(sorted(additional_vars))}"

predefined_sample_set_filename += (f"_{data_type_str}_{sel_disease_type_str}_"
                                   f"{'-'.join(map(str, min_num_samples_per_type_cls))}")
predefined_sample_set_filename += f"_{'-'.join(map(str, [train_portion, val_portion, test_portion]))}"
if cv_type == 'group-shuffle' and num_train_types > 0:
  predefined_sample_set_filename += f"_{'-'.join(map(str, [num_train_types, num_val_types, num_test_types]))}"
predefined_sample_set_filename += f'_{num_sets}sets'
res_file = f"{predefined_sample_set_filename}_{sel_set_idx}_{'-'.join(map(str, hidden_dim))}_{model_type}.pkl"
predefined_sample_set_filename += '.pkl'
# This will be overwritten if predefined_sample_set_file == 'auto-search' or filepath, and the file exists
predefined_sample_sets = [get_shuffled_data(sel_patient_ids, clinical_dict, cv_type=cv_type, 
                  instance_portions=[train_portion, val_portion, test_portion], 
                  group_sizes=[num_train_types, num_val_types, num_test_types],
                  group_variable_name='type', seed=None, verbose=False) for i in range(num_sets)]
if predefined_sample_set_file == 'auto-search':
  if os.path.exists(f'{data_split_idx_folder}/{predefined_sample_set_filename}'):
    with open(f'{data_split_idx_folder}/{predefined_sample_set_filename}', 'rb') as f:
      print(f'Read predefined_sample_set_file: '
            f'{data_split_idx_folder}/{predefined_sample_set_filename}')
      tmp = pickle.load(f)
      # overwrite calculated predefined_sample_sets
      predefined_sample_sets = tmp['predefined_sample_sets']    
elif isinstance(predefined_sample_set_file, str): # but not 'auto-search'; assume it's a file name
  if os.path.exists(predefined_sample_set_file):
    with open(f'{data_split_idx_folder}/{predefined_sample_set_file}', 'rb') as f:
      print(f'Read predefined_sample_set_file: {data_split_idx_folder}/{predefined_sample_set_file}')
      tmp = pickle.load(f)
      predefined_sample_sets = tmp['predefined_sample_sets']
  else:
    raise ValueError(f'predefined_sample_set_file: {data_split_idx_folder}/{predefined_sample_set_file} does not exist!')

if (not os.path.exists(f'{data_split_idx_folder}/{predefined_sample_set_filename}') 
    and predefined_sample_set_file == 'auto-search') or predefined_sample_set_file is True:
  with open(f'{data_split_idx_folder}/{predefined_sample_set_filename}', 'wb') as f:
      print(f'Write predefined_sample_set_file: {data_split_idx_folder}/{predefined_sample_set_filename}')
      pickle.dump({'predefined_sample_sets': predefined_sample_sets}, f)
     
sel_patient_ids, idx_splits = predefined_sample_sets[sel_set_idx]
train_idx, val_idx, test_idx = idx_splits

Read predefined_sample_set_file: results/data_split_idx/PFI-DFI-PFI.time_instance-shuffle_age_at_initial_pathologic_diagnosis-ajcc_pathologic_tumor_stage-gender_gene-methy-mirna-rppa_all_100-0_0.7-0.1-0.2_10sets.pkl


In [6]:
if isinstance(data_type, str):
  sample_lists = [aliquot_id_dict[data_type]]
else:
  assert isinstance(data_type, (list, tuple))
  sample_lists = [aliquot_id_dict[dtype] for dtype in data_type]
idx_lists = get_overlap_samples(sample_lists=sample_lists, common_list=sel_patient_ids, 
                    start=0, end=12, return_common_list=False)
sample_idx_sel_dict = {}
if isinstance(data_type, str):
  sample_idx_sel_dict = {data_type: idx_lists[0]}
else:
  sample_idx_sel_dict = {dtype: idx_list for dtype, idx_list in zip(data_type, idx_lists)}

In [7]:
if isinstance(data_type, str):
  print(f'Only use one data type: {data_type}')
  num_data_types = 1
  mat = feature_mat_dict[data_type][sample_idx_sel_dict[data_type]]
  # Data preprocessing: make each row have mean 0 and sd 1.
  x = (mat - mat.mean(axis=1, keepdims=True)) / mat.std(axis=1, keepdims=True)
  interaction_mat = feature_interaction_mat_dict[data_type]
  interaction_mat = torch.from_numpy(interaction_mat).float().to(device)
  # Normalize these interaction mat
  interaction_mat = interaction_mat / interaction_mat.norm()
else:
  mat = []
  interaction_mats = []
  in_dims = []
  num_data_types = len(data_type)
  # do not handle the special case of [data_type] to avoid too much code complexity
  assert num_data_types > 1 
  for dtype in data_type: # multiple data types
    m = feature_mat_dict[dtype][sample_idx_sel_dict[dtype]]
    #When there are multiple data types, make sure each type is normalized to have mean 0 and std 1
    m = (m - m.mean(axis=1, keepdims=True)) / m.std(axis=1, keepdims=True)
    mat.append(m)
    in_dims.append(m.shape[1])
    # For neural network model graph laplacian regularizer
    interaction_mat = feature_interaction_mat_dict[dtype]
    interaction_mat = torch.from_numpy(interaction_mat).float().to(device)
    # Normalize these interaction mat
    interaction_mat = interaction_mat / interaction_mat.norm()
    interaction_mats.append(interaction_mat)
    print(f'{dtype}: {m.shape}; '
          f'interaction_mat: mean={interaction_mat.mean().item():2f}, '
          f'std={interaction_mat.std().item():2f}, {interaction_mat.shape[0]}')
  # Later interaction_mat will be passed to Loss_feature_interaction
  interaction_mat = interaction_mats
  mat = np.concatenate(mat, axis=1)
  # For machine learing methods that use concatenated features without knowing underlying views,
  # it might be good to make each row have mean 0 and sd 1.
  x = (mat - mat.mean(axis=1, keepdims=True)) / mat.std(axis=1, keepdims=True)

if normal_transform_feature:
  X = x
else:
  X = mat

gene: (2083, 4942); interaction_mat: mean=0.000057, std=0.000194, 4942
methy: (2083, 4753); interaction_mat: mean=0.000069, std=0.000199, 4753
rppa: (2083, 189); interaction_mat: mean=0.002668, std=0.004569, 189
mirna: (2083, 662); interaction_mat: mean=0.001408, std=0.000547, 662


In [8]:
# sklearn classifiers also accept torch.Tensor
X = torch.tensor(X).float().to(device)
x_train = X[train_idx]
x_val = X[val_idx]
x_test = X[test_idx]
print(x_train.shape, x_val.shape, x_test.shape)

torch.Size([1458, 10546]) torch.Size([208, 10546]) torch.Size([417, 10546])


In [12]:
y_targets = get_target_variable(target_variable, clinical_dict, sel_patient_ids)
y_targets = normalize_continuous_variable(y_targets, target_variable_type, transform=True, 
                                          forced=False, threshold=10, rm_outlier=True, whis=1.5, 
                                          only_positive=True, max_val=1)
y_true = target_to_numpy(y_targets, target_variable_type, target_variable_range)
if len(additional_vars) > 0:
  additional_variables = get_target_variable(additional_vars, clinical_dict, sel_patient_ids)
  # to do handle additional variables such as age and gender

# should have written a recursive function instead
if isinstance(target_variable_type, list):
  y_targets = []
  num_cls = []
  y_train = []
  y_val = []
  y_test = []
  for i, var_type in enumerate(target_variable_type):
    y = torch.tensor(y_true[i]).to(device)
    if var_type == 'discrete':
      y = y.long()
    elif var_type == 'continuous':
      y = y.float()
      if y.dim()==1:
        y = y.unsqueeze(-1)
    else:
      raise ValueError(f'target type should be either discrete or continuous but is {var_type}')
    y_targets.append(y)
    num_cls.append(len(torch.unique(y))) # include continous target variables
    y_train.append(y[train_idx])
    y_val.append(y[val_idx])
    y_test.append(y[test_idx])
    print(f'{target_variable[i]}:\ntrain:{y_train[-1].shape}, val:{y_val[-1].shape}, '
         f'test:{y_test[-1].shape}')
    if var_type == 'discrete':
      label_probs = get_label_distribution([y_train[-1], y_val[-1], y_test[-1]])
      if randomize_labels: # Optionally randomize true class labels
        print('Randomize class labels!')
        y_train[-1] = torch.multinomial(label_probs[0], len(y_train[-1]), replacement=True)
        if len(y_val) > 0:
          y_val[-1] = torch.multinomial(label_probs[1], len(y_val[-1]), replacement=True)
        if len(y_test) > 0:
          y_test[-1] = torch.multinomial(label_probs[2], len(y_test[-1]), replacement=True)
        get_label_distribution([y_train[-1], y_val[-1], y_test[-1]])
  y_true = y_targets
elif isinstance(target_variable_type, str):
  y = torch.tensor(y_true).to(device)
  if var_type == 'discrete':
    y = y.long()
  elif var_type == 'continuous':
    y = y.float()
    if y.dim()==1:
      y = y.unsqueeze(-1)
  else:
    raise ValueError(f'target type should be either discrete or continuous but is {var_type}')
  y_true = y
  num_cls = len(torch.unique(y_true))
  y_train = y_true[train_idx]
  y_val = y_true[val_idx]
  y_test = y_true[test_idx]
  print(f'{target_variable}:\ntrain:{y_train.shape}, val:{y_val.shape}, '
         f'test:{y_test.shape}')
  label_probs = get_label_distribution([y_train, y_val, y_test])
  if randomize_labels: # Optionally randomize true class labels
    print('Randomize class labels!')
    y_train = torch.multinomial(label_probs[0], len(y_train), replacement=True)
    if len(y_val) > 0:
      y_val = torch.multinomial(label_probs[1], len(y_val), replacement=True)
    if len(y_test) > 0:
      y_test = torch.multinomial(label_probs[2], len(y_test), replacement=True)
    get_label_distribution([y_train, y_val, y_test])
else:
  raise ValueError(f'target_variable_type should be str or list, but is {type(target_variable_type)}')

Changed class labels for the model: {0.0: 0, 1.0: 1}
Changed class labels for the model: {0.0: 0, 1.0: 1}
PFI:
train:torch.Size([1458]), val:torch.Size([208]), test:torch.Size([417])
label distribution:
 [[0.82304525 0.84615386 0.81534773]
 [0.17695473 0.15384616 0.18465228]]
DFI:
train:torch.Size([1458]), val:torch.Size([208]), test:torch.Size([417])
label distribution:
 [[0.8360768  0.84615386 0.82254195]
 [0.16392319 0.15384616 0.17745803]]
PFI.time:
train:torch.Size([1458, 1]), val:torch.Size([208, 1]), test:torch.Size([417, 1])


## Use additional variables for prediction

In [13]:
embedding_dim = 50
input_list = []
xs = []
for v, n, t in zip(additional_variables, additional_vars, additional_var_types):
  if n.startswith('age'):    
    bins = [0, 20, 30, 40, 50, 60, 70, 80, 100]
    v = np.digitize(v, bins)
    t = 'discrete'
  if t=='discrete':
    target_ids, cls_id_dict = discrete_to_id(v, start=0, sort=True)
    # some target_ids may have very few instances
    print(n)
    print(collections.Counter(target_ids))
    xs.append(torch.tensor(target_ids, device=device).long())
    # did not handle missing value yet
    input_list.append({'in_dim': len(cls_id_dict), 'in_type': 'discrete', 'padding_idx':None, 
                       'embedding_dim':embedding_dim, 'hidden_dim': hidden_dim})
  else: # t=='continuous'
    xs.append(torch.tensor(v, device=device).float())
    input_list.append({'in_dim': len(v[0]), 'in_type': 'continuous', 'hidden_dim': hidden_dim})

age_at_initial_pathologic_diagnosis
Counter({5: 577, 4: 468, 6: 429, 3: 285, 7: 137, 2: 130, 1: 48, 0: 9})
gender
Counter({0: 1322, 1: 761})
ajcc_pathologic_tumor_stage
Counter({0: 398, 4: 379, 5: 276, 8: 220, 3: 185, 1: 164, 7: 162, 2: 144, 10: 77, 9: 76, 11: 1, 6: 1})


In [15]:
if isinstance(target_variable, list):
  for i, var_name in enumerate(target_variable):
    if target_variable_type[i]=='discrete':
      print(f'target: {var_name}')
      get_mi_acc(xs, y_true=y_true[i], var_names=additional_vars, var_name_length=35)
elif isinstance(target_variable, str):
  if target_variable_type=='discrete':
    print(f'target: {target_variable}')
    get_mi_acc(xs, y_true, var_names=additional_vars, var_name_length=35)

target: PFI
             Variable              	 MI  	Adj_MI	Bayes_ACC
age_at_initial_pathologic_diagnosis	0.003	0.001 	  0.824  
              gender               	0.009	0.013 	  0.824  
    ajcc_pathologic_tumor_stage    	0.012	0.004 	  0.824  
                0-1                	0.011	0.003 	  0.824  
                0-2                	0.029	0.003 	  0.825  
                1-2                	0.033	0.010 	  0.831  
               0-1-2               	0.061	0.006 	  0.836  
target: DFI
             Variable              	 MI  	Adj_MI	Bayes_ACC
age_at_initial_pathologic_diagnosis	0.002	0.000 	  0.834  
              gender               	0.009	0.013 	  0.834  
    ajcc_pathologic_tumor_stage    	0.011	0.004 	  0.835  
                0-1                	0.011	0.003 	  0.834  
                0-2                	0.028	0.003 	  0.836  
                1-2                	0.029	0.009 	  0.838  
               0-1-2               	0.056	0.005 	  0.843  


In [16]:
xs_train = [x[train_idx] for x in xs]
xs_val = [x[val_idx] for x in xs]
xs_test = [x[test_idx] for x in xs]

In [17]:
last_nonlinearity = True
input_list.append({'in_dim': x_train.size(1), 'in_type': 'continuous', 'hidden_dim': hidden_dim,
                  'last_nonlinearity':last_nonlinearity, })
xs_train.append(x_train)
xs_val.append(x_val)
xs_test.append(x_test)

for i in xs_train:
  print(i.shape)

torch.Size([1458])
torch.Size([1458])
torch.Size([1458])
torch.Size([1458, 10546])


In [19]:
hidden_dim = [100]
output_info = [{'hidden_dim': hidden_dim+[2]}, {'hidden_dim': hidden_dim+[2]},
              {'hidden_dim': hidden_dim+[1]}]
            
fusion_lists = [[{'fusion_type': 'repr-weighted-avg_repr', 'hidden_dim': hidden_dim, 
                  'last_nonlinearity':last_nonlinearity, 'output_info': output_info}, 
                 {'fusion_type': 'repr-loss-avg_repr', 'hidden_dim': hidden_dim, 
                  'last_nonlinearity':last_nonlinearity, 'output_info': output_info},
                 {'fusion_type': 'repr-avg_repr', 'hidden_dim': hidden_dim, 
                  'last_nonlinearity':last_nonlinearity, 'output_info': output_info},
                 {'fusion_type': 'repr-cat_repr', 'hidden_dim': hidden_dim, 
                  'last_nonlinearity':last_nonlinearity, 'output_info': output_info},
                 {'fusion_type': 'out-weighted-avg', 'output_info': output_info},
                 {'fusion_type': 'out-loss-avg', 'output_info': output_info},
                 {'fusion_type': 'out-avg', 'output_info': output_info}],
                [{'fusion_type': 'repr-weighted-avg_repr', 'hidden_dim': hidden_dim, 
                  'last_nonlinearity':last_nonlinearity, 'output_info': output_info}, 
                 {'fusion_type': 'repr-loss-avg_repr', 'hidden_dim': hidden_dim, 
                  'last_nonlinearity':last_nonlinearity, 'output_info': output_info},
                 {'fusion_type': 'repr-avg_repr', 'hidden_dim': hidden_dim, 
                  'last_nonlinearity':last_nonlinearity, 'output_info': output_info},
                 {'fusion_type': 'repr-cat_repr', 'hidden_dim': hidden_dim, 
                  'last_nonlinearity':last_nonlinearity, 'output_info': output_info},
                 {'fusion_type': 'out-weighted-avg', 'output_info': output_info},
                 {'fusion_type': 'out-loss-avg', 'output_info': output_info},
                 {'fusion_type': 'out-avg', 'output_info': output_info}],
                [{'fusion_type': 'repr0', 'hidden_dim': hidden_dim, 
                  'last_nonlinearity':last_nonlinearity, 'output_info': output_info}]
               ]

In [20]:
model = VIN(input_list, output_info, fusion_lists, nonlinearity=nn.ReLU())
if target_variable_type=='discrete':
  loss_fn = nn.CrossEntropyLoss()
elif target_variable_type=='dontinuous':
  loss_fn = nn.MSELoss()
else:
  loss_fn = []
  for var_type in target_variable_type:
    if var_type == 'discrete':
      loss_fn.append(nn.CrossEntropyLoss())
    elif var_type == 'continuous':
      loss_fn.append(nn.MSELoss())
    else:
      raise ValueError(f'target type should be either discrete or continous, but is {var_type}')

In [21]:
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), 
                             lr=1e-2, weight_decay=weight_decay, amsgrad=True)

In [22]:
for n, p in model.named_parameters():
#   print(n, p.size())
  if p.grad is not None and p.grad.norm()==0:
    print(n, p.grad if p.grad is None else p.grad.norm())

In [23]:
target_loss_weight = [1., 1., 1.]

In [24]:
loss_his = []
num_iters = 100
print_every = 100
for i in range(num_iters):
  pred = model(xs_test)
  losses = get_vin_loss(pred, y_test, loss_fn, model, valid_loc=None, target_id=None, 
                        level_weight=None)
  loss = sum(losses[j][0]*target_loss_weight[j] for j in range(len(losses)))
  losses = losses[0]
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  loss_his.append([losses[0].item()] + 
                [[[v.item() for v in losses[1][i][0]], losses[1][i][1].item()] for i in range(2)])
  if i%print_every == 0 or i==num_iters-1:
    print(i, loss.item())

0 6.476935386657715
99 1.0573850870132446


In [25]:
pred = model(xs_test)
eval_classification(y_test[0], pred[0][-1][0])
get_vin_loss(pred, y_test, loss_fn, model, valid_loc=None, target_id=None, 
                        level_weight=None)

acc=1.000, precision=1.000, recall=1.000, fl=1.000, adj_MI=1.000, auc=1.000, ap=1.000, confusion_mat=
[[340   0]
 [  0  77]]
report              precision    recall  f1-score   support

          0       1.00      1.00      1.00       340
          1       1.00      1.00      1.00        77

avg / total       1.00      1.00      1.00       417



[(tensor(0.3750, grad_fn=<ThAddBackward>),
  [[[tensor(0.4638, grad_fn=<NllLossBackward>),
     tensor(0.4475, grad_fn=<NllLossBackward>),
     tensor(0.4651, grad_fn=<NllLossBackward>),
     tensor(0.0441, grad_fn=<NllLossBackward>)],
    tensor(0.3546, grad_fn=<ThAddBackward>)],
   [[tensor(0.0055, grad_fn=<NllLossBackward>),
     tensor(0.0074, grad_fn=<NllLossBackward>),
     tensor(0.0078, grad_fn=<NllLossBackward>),
     tensor(0.0036, grad_fn=<NllLossBackward>),
     tensor(0.0263, grad_fn=<NllLossBackward>),
     tensor(0.0265, grad_fn=<NllLossBackward>),
     tensor(0.0267, grad_fn=<NllLossBackward>)],
    tensor(0.0128, grad_fn=<ThAddBackward>)],
   [[tensor(0.0036, grad_fn=<NllLossBackward>),
     tensor(0.0036, grad_fn=<NllLossBackward>),
     tensor(0.0037, grad_fn=<NllLossBackward>),
     tensor(0.0031, grad_fn=<NllLossBackward>),
     tensor(0.0083, grad_fn=<NllLossBackward>),
     tensor(0.0080, grad_fn=<NllLossBackward>),
     tensor(0.0091, grad_fn=<NllLossBackward>)]

In [26]:
pred = model(xs_train)
eval_classification(y_train[0], pred[0][-1][0])
get_vin_loss(pred, y_train, loss_fn, model, valid_loc=None, target_id=None, 
                   level_weight=None)

acc=0.765, precision=0.736, recall=0.765, fl=0.749, adj_MI=0.007, auc=0.627, ap=0.253, confusion_mat=
[[1067  133]
 [ 209   49]]
report              precision    recall  f1-score   support

          0       0.84      0.89      0.86      1200
          1       0.27      0.19      0.22       258

avg / total       0.74      0.77      0.75      1458



[(tensor(6.2221, grad_fn=<ThAddBackward>),
  [[[tensor(0.4945, grad_fn=<NllLossBackward>),
     tensor(0.4714, grad_fn=<NllLossBackward>),
     tensor(0.4632, grad_fn=<NllLossBackward>),
     tensor(2.8643, grad_fn=<NllLossBackward>)],
    tensor(1.0757, grad_fn=<ThAddBackward>)],
   [[tensor(1.6178, grad_fn=<NllLossBackward>),
     tensor(1.8301, grad_fn=<NllLossBackward>),
     tensor(1.5669, grad_fn=<NllLossBackward>),
     tensor(1.8302, grad_fn=<NllLossBackward>),
     tensor(0.8695, grad_fn=<NllLossBackward>),
     tensor(0.8655, grad_fn=<NllLossBackward>),
     tensor(0.8631, grad_fn=<NllLossBackward>)],
    tensor(1.4329, grad_fn=<ThAddBackward>)],
   [[tensor(1.7378, grad_fn=<NllLossBackward>),
     tensor(1.7598, grad_fn=<NllLossBackward>),
     tensor(1.7488, grad_fn=<NllLossBackward>),
     tensor(2.0182, grad_fn=<NllLossBackward>),
     tensor(1.3899, grad_fn=<NllLossBackward>),
     tensor(1.4158, grad_fn=<NllLossBackward>),
     tensor(1.3304, grad_fn=<NllLossBackward>)]

In [27]:
eval_classification_multi_splits(model, [xs_train, xs_val, xs_test], [y_train[0], y_val[0], y_test[0]], 
  batch_size=None, multi_heads=False, cls_head=0, average='weighted', return_result=True, 
  split_names=['Train', 'Validataion', 'Test'], verbose=True, 
  predict_func=predict_func, pred_kwargs={'target_idx':0, 'level':-1, 'loc':0, 'train':False})

Train
acc=0.765, precision=0.736, recall=0.765, fl=0.749, adj_MI=0.007, auc=0.627, ap=0.253, confusion_mat=
[[1067  133]
 [ 209   49]]
report              precision    recall  f1-score   support

          0       0.84      0.89      0.86      1200
          1       0.27      0.19      0.22       258

avg / total       0.74      0.77      0.75      1458

Validataion
acc=0.808, precision=0.798, recall=0.808, fl=0.802, adj_MI=0.041, auc=0.675, ap=0.315, confusion_mat=
[[158  18]
 [ 22  10]]
report              precision    recall  f1-score   support

          0       0.88      0.90      0.89       176
          1       0.36      0.31      0.33        32

avg / total       0.80      0.81      0.80       208

Test
acc=1.000, precision=1.000, recall=1.000, fl=1.000, adj_MI=1.000, auc=1.000, ap=1.000, confusion_mat=
[[340   0]
 [  0  77]]
report              precision    recall  f1-score   support

          0       1.00      1.00      1.00       340
          1       1.00      1.00      1.

[(array([0.7654321 , 0.73587779, 0.7654321 , 0.74877395, 0.00738225,
         0.62744186, 0.25277694]), array([[1067,  133],
         [ 209,   49]], dtype=int64)),
 (array([0.80769231, 0.7976801 , 0.80769231, 0.80236243, 0.04149792,
         0.67542614, 0.31500486]), array([[158,  18],
         [ 22,  10]], dtype=int64)),
 (array([1., 1., 1., 1., 1., 1., 1.]), array([[340,   0],
         [  0,  77]], dtype=int64))]

## Neural network models

In [ ]:
# loss_fn_cls = torch.nn.CrossEntropyLoss(weight=torch.tensor([0.3, 0.6], device=device))
loss_fn_cls = torch.nn.CrossEntropyLoss()
loss_fn_reg = torch.nn.MSELoss()
loss_fns = [loss_fn_cls, loss_fn_reg]
# For multiple data types, there are multiple interaction mats
feat_interact_loss_type = 'graph_laplacian'
if num_data_types > 1:
  weight_path = ['decoders', range(num_data_types), 'weight']  
else:
  weight_path = ['decoder', 'weight']
loss_feat_interact = Loss_feature_interaction(interaction_mat=interaction_mat, 
                                              loss_type=feat_interact_loss_type, 
                                              weight_path=weight_path, 
                                              normalize=True)
other_loss_fns = [loss_feat_interact]
if num_data_types > 1:
  view_sim_loss_type = 'hub'
  explicit_target = True
  cal_target='mean-feature'
  # In this set of experiments, the encoders for all views will have the same hidden_dim
  loss_view_sim = Loss_view_similarity(sections=hidden_dim[-1], loss_type=view_sim_loss_type, 
    explicit_target=explicit_target, cal_target=cal_target, target=None)
  loss_fns.append(loss_view_sim)

In [ ]:
model_names = []
split_names = ['train', 'val', 'test']
metric_names = ['acc', 'precision', 'recall', 'f1_score', 'adjusted_mutual_info', 'auc', 
                'average_precision']
metric_all = []
confusion_mat_all = []
loss_his_all = []
acc_his_all = []

In [ ]:
def get_result(model, best_model, best_val_acc, best_epoch, x_train, y_train, x_val, y_val, 
               x_test, y_test, batch_size, multi_heads, show_results_in_notebook=True, 
               loss_idx=0, acc_idx=0):
  if len(x_val) > 0:
    print(f'Best model on validation set: best_val_acc={best_val_acc:.2f}, epoch={best_epoch}')
    metric = eval_classification_multi_splits(best_model, xs=[x_train, x_val, x_test], 
      ys=[y_train, y_val, y_test], batch_size=batch_size, multi_heads=multi_heads)

  if show_results_in_notebook:
    print('\nModel after the last training epoch:')
    eval_classification_multi_splits(model, xs=[x_train, x_val, x_test], 
                                     ys=[y_train, y_val, y_test], batch_size=batch_size, 
                                     multi_heads=multi_heads, return_result=False)

    plot_history_multi_splits([loss_train_his, loss_val_his, loss_test_his], title='Loss', 
                              idx=loss_idx)
    plot_history_multi_splits([acc_train_his, acc_val_his, acc_test_his], title='Acc', idx=acc_idx)
    # scatter plot
    plot_data_multi_splits(best_model, [x_train, x_val, x_test], [y_train, y_val, y_test], 
                           num_heads=2 if multi_heads else 1, 
                           titles=['Training', 'Validation', 'Test'], batch_size=batch_size)
    return metric

# Plain deep learning model

In [ ]:
batch_size = 1000
print_every = 100
eval_every = 1

In [ ]:
in_dim = x_train.shape[1]
print('Plain deep learning model')
model_names.append('NN')
model = DenseLinear(in_dim, hidden_dim+[num_cls], dense=dense, residual=residual).to(device)
multi_heads = False

loss_train_his = []
loss_val_his = []
loss_test_his = []
acc_train_his = []
acc_val_his = []
acc_test_his = []
best_model = model
best_val_acc = 0
best_epoch = 0

In [ ]:
best_model, best_val_acc, best_epoch = train_single_loss(model, x_train, y_train, 
    x_val, y_val, x_test, y_test, loss_fn=loss_fn_cls, lr=lr, weight_decay=weight_decay, 
    amsgrad=True, batch_size=batch_size, num_epochs=num_epochs, 
    reduce_every=reduce_every, eval_every=eval_every, print_every=print_every, verbose=False, 
    loss_train_his=loss_train_his, loss_val_his=loss_val_his, loss_test_his=loss_test_his, 
    acc_train_his=acc_train_his, acc_val_his=acc_val_his, acc_test_his=acc_test_his, 
    return_best_val=True)

In [ ]:
metric = get_result(model, best_model, best_val_acc, best_epoch, x_train, y_train, x_val, y_val, 
                    x_test, y_test, batch_size, multi_heads, show_results_in_notebook, 
                    loss_idx=0, acc_idx=0)

In [ ]:
loss_his_all.append([loss_train_his, loss_val_his, loss_test_his])
acc_his_all.append([acc_train_his, acc_val_his, acc_test_his])
metric_all.append([v[0] for v in metric])
confusion_mat_all.append([v[1] for v in metric])

# Factorization AutoEncoder

In [ ]:
def run_one_model(model, loss_weights, other_loss_weights, 
                  loss_his_all=[], acc_his_all=[], metric_all=[], confusion_mat_all=[],
                  heads=[0,1], multi_heads=True, return_results=False, 
                  loss_fns=loss_fns, other_loss_fns=other_loss_fns, 
                  lr=lr, weight_decay=weight_decay, batch_size=batch_size, 
                  num_epochs=num_epochs, reduce_every=reduce_every, eval_every=eval_every, 
                  print_every=print_every, x_train=x_train, y_train=y_train,
                  x_val=x_val, y_val=y_val, x_test=x_test, y_test=y_test,
                  show_results_in_notebook=show_results_in_notebook):
  """Train a model and get results  
    Most of the parameters are from the context; handle it properly
  """
  loss_train_his = []
  loss_val_his = []
  loss_test_his = []
  acc_train_his = []
  acc_val_his = []
  acc_test_his = []
  best_model = model
  best_val_acc = 0
  best_epoch = 0

  best_model, best_val_acc, best_epoch = train_multiloss(model, x_train, [y_train, x_train], 
    x_val, [y_val, x_val], x_test, [y_test, x_test], heads=heads, loss_fns=loss_fns, 
    loss_weights=loss_weights, other_loss_fns=other_loss_fns, 
    other_loss_weights=other_loss_weights, 
    lr=lr, weight_decay=weight_decay, batch_size=batch_size, num_epochs=num_epochs, 
    reduce_every=reduce_every, eval_every=eval_every, print_every=print_every,
    loss_train_his=loss_train_his, loss_val_his=loss_val_his, loss_test_his=loss_test_his, 
    acc_train_his=acc_train_his, acc_val_his=acc_val_his, acc_test_his=acc_test_his, 
    return_best_val=True, amsgrad=True, verbose=False)

  metric = get_result(model, best_model, best_val_acc, best_epoch, x_train, y_train, x_val, y_val, 
                      x_test, y_test, batch_size, multi_heads, show_results_in_notebook, 
                      loss_idx=0, acc_idx=0)

  loss_his_all.append([loss_train_his, loss_val_his, loss_test_his])
  acc_his_all.append([acc_train_his, acc_val_his, acc_test_his])
  metric_all.append([v[0] for v in metric])
  confusion_mat_all.append([v[1] for v in metric])
  
  if return_results:
    return loss_his_all, acc_his_all, metric_all, confusion_mat_all

In [ ]:
decoder_norm = False
uniform_decoder_norm = False
print('Plain AutoEncoder model')
model_names.append('AE')
model = AutoEncoder(in_dim, hidden_dim, num_cls, dense=dense, residual=residual,
          decoder_norm=decoder_norm, uniform_decoder_norm=uniform_decoder_norm).to(device)
loss_weights = [1,1]
other_loss_weights = [0]
# heads = None should work for all the following; keep this for clarity
heads = [0,1] 
run_one_model(model, loss_weights, other_loss_weights,
              loss_his_all, acc_his_all, metric_all, confusion_mat_all,
              heads=heads, multi_heads=True, return_results=False, 
              loss_fns=loss_fns, other_loss_fns=other_loss_fns, 
              lr=lr, weight_decay=weight_decay, batch_size=batch_size, 
              num_epochs=num_epochs, reduce_every=reduce_every, eval_every=eval_every, 
              print_every=print_every, x_train=x_train, y_train=y_train,
              x_val=x_val, y_val=y_val, x_test=x_test, y_test=y_test,
              show_results_in_notebook=show_results_in_notebook)

## Add feature interaction network regularizer

In [ ]:
if num_data_types > 1:
  fuse_type = 'sum'
  print('MultiviewAE with feature interaction network regularizer')
  model_names.append('MultiviewAE + feat_int')
  model = MultiviewAE(in_dims=in_dims, hidden_dims=hidden_dim, out_dim=num_cls, 
                      fuse_type=fuse_type, dense=dense, residual=residual, 
                      residual_layers='all', decoder_norm=decoder_norm, 
                      decoder_norm_dim=0, uniform_decoder_norm=uniform_decoder_norm, 
                      nonlinearity=nn.ReLU(), last_nonlinearity=True, bias=True).to(device)
else:
  print('AutoEncoder with feature interaction network regularizer')
  model_names.append('AE + feat_int')
  model = AutoEncoder(in_dim, hidden_dim, num_cls, dense=dense, residual=residual, 
          decoder_norm=decoder_norm, uniform_decoder_norm=uniform_decoder_norm).to(device)

loss_weights = [1,1]
other_loss_weights = [1]
heads = [0,1]
run_one_model(model, loss_weights, other_loss_weights, 
              loss_his_all, acc_his_all, metric_all, confusion_mat_all,
              heads=heads, multi_heads=True, return_results=False, 
              loss_fns=loss_fns, other_loss_fns=other_loss_fns, 
              lr=lr, weight_decay=weight_decay, batch_size=batch_size, 
              num_epochs=num_epochs, reduce_every=reduce_every, eval_every=eval_every, 
              print_every=print_every, x_train=x_train, y_train=y_train,
              x_val=x_val, y_val=y_val, x_test=x_test, y_test=y_test,
              show_results_in_notebook=show_results_in_notebook)

## For multi-view data, add view similarity network regularizer

In [ ]:
if num_data_types > 1:
  # plain multiviewAE; compare it with plain AutoEncoder to see 
  # if separating views in lower layers in MultiviewAE is better than combining them all the way
  print('Run plain MultiviewAE model')
  model_names.append('MultiviewAE')
  model = MultiviewAE(in_dims=in_dims, hidden_dims=hidden_dim, out_dim=num_cls, 
                    fuse_type=fuse_type, dense=dense, residual=residual, 
                    residual_layers='all', decoder_norm=decoder_norm, 
                    decoder_norm_dim=0, uniform_decoder_norm=uniform_decoder_norm, 
                    nonlinearity=nn.ReLU(), last_nonlinearity=True, bias=True).to(device)

  loss_weights = [1,1]
  other_loss_weights = [0]
  heads = [0,1]
  run_one_model(model, loss_weights, other_loss_weights, 
                loss_his_all, acc_his_all, metric_all, confusion_mat_all,
                heads=heads, multi_heads=True, return_results=False, 
                loss_fns=loss_fns, other_loss_fns=other_loss_fns, 
                lr=lr, weight_decay=weight_decay, batch_size=batch_size, 
                num_epochs=num_epochs, reduce_every=reduce_every, eval_every=eval_every, 
                print_every=print_every, x_train=x_train, y_train=y_train,
                x_val=x_val, y_val=y_val, x_test=x_test, y_test=y_test,
                show_results_in_notebook=show_results_in_notebook)

In [ ]:
if num_data_types > 1:
  print('MultiviewAE with view similarity regularizers')
  model_names.append('MultiviewAE + view_sim')
  model = MultiviewAE(in_dims=in_dims, hidden_dims=hidden_dim, out_dim=num_cls, 
                      fuse_type=fuse_type, dense=dense, residual=residual, 
                      residual_layers='all', decoder_norm=decoder_norm, 
                      decoder_norm_dim=0, uniform_decoder_norm=uniform_decoder_norm, 
                      nonlinearity=nn.ReLU(), last_nonlinearity=True, bias=True).to(device)
  loss_weights = [1,1,1]
  other_loss_weights = [0]
  heads = [0,1,2]
  run_one_model(model, loss_weights, other_loss_weights, 
                loss_his_all, acc_his_all, metric_all, confusion_mat_all,
                heads=heads, multi_heads=True, return_results=False, 
                loss_fns=loss_fns, other_loss_fns=other_loss_fns, 
                lr=lr, weight_decay=weight_decay, batch_size=batch_size, 
                num_epochs=num_epochs, reduce_every=reduce_every, eval_every=eval_every, 
                print_every=print_every, x_train=x_train, y_train=y_train,
                x_val=x_val, y_val=y_val, x_test=x_test, y_test=y_test,
                show_results_in_notebook=show_results_in_notebook)

In [ ]:
if num_data_types > 1:
  print('MultiviewAE with both feature interaction and view similarity regularizers')
  model_names.append('MultiviewAE + feat_int + view_sim')
  model = MultiviewAE(in_dims=in_dims, hidden_dims=hidden_dim, out_dim=num_cls, 
                      fuse_type=fuse_type, dense=dense, residual=residual, 
                      residual_layers='all', decoder_norm=decoder_norm, 
                      decoder_norm_dim=0, uniform_decoder_norm=uniform_decoder_norm, 
                      nonlinearity=nn.ReLU(), last_nonlinearity=True, bias=True).to(device)
  loss_weights = [1,1,1]
  other_loss_weights = [1]
  heads = [0,1,2]
  run_one_model(model, loss_weights, other_loss_weights,
                loss_his_all, acc_his_all, metric_all, confusion_mat_all,
                heads=heads, multi_heads=True, return_results=False, 
                loss_fns=loss_fns, other_loss_fns=other_loss_fns, 
                lr=lr, weight_decay=weight_decay, batch_size=batch_size, 
                num_epochs=num_epochs, reduce_every=reduce_every, eval_every=eval_every, 
                print_every=print_every, x_train=x_train, y_train=y_train,
                x_val=x_val, y_val=y_val, x_test=x_test, y_test=y_test,
                show_results_in_notebook=show_results_in_notebook)

In [ ]:
with open(f'{result_folder}/{res_file}', 'wb') as f:
  print(f'Write result to file {result_folder}/{res_file}')
  pickle.dump({'loss_his_all': loss_his_all,
               'acc_his_all': acc_his_all,
               'metric_all': metric_all,
               'confusion_mat_all': confusion_mat_all,
               'model_names': model_names,
               'split_names': split_names,
               'metric_names': metric_names
              }, f)